# Recommender System Overview
This project implements a movie recommender system using the following techniques:

- **Search Method (SI):** Firefly Algorithm
- **Dataset:** MovieLens
- **Similarity Measure:** Pearson Correlation Coefficient
- **Rating Prediction Method:** Basic K-Nearest Neighbors (KNN)

# Evaluation Metrics for Recommendation Systems
In this project, we evaluate the performance of the recommender system using the following metrics:

- **Precision**
- **Accuracy**
- **F-Measure (F1-Score)**
- **Recall**

These metrics help assess how well the system recommends relevant movies and how well it balances precision (accuracy of recommendations) with recall (coverage of relevant recommendations).

## Define the libraries 

In [1]:
from math import sqrt
import pandas as pd 
import numpy as np
import seaborn as sns
import random
import sys
import math, copy
import csv
import surprise
import os
import io
from collections import defaultdict
from numpy import inf
from itertools import groupby
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise import KNNBasic
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import pairwise_distances
from surprise.model_selection import cross_validate

## Define the datasets

In [9]:
df_ratings = pd.read_csv("datasets/rating.csv")

df_movies = pd.read_csv("datasets/movie.csv")

df_movies_ratings=pd.concat([df_movies, df_ratings], axis='columns')
df_movies_ratings = df_movies_ratings[['movieId','title','userId','rating']]
df_movies_ratings.head()

,movieId,movieId,title,userId,rating
0,1.0,2,Toy Story (1995),1,3.5
1,2.0,29,Jumanji (1995),1,3.5
2,3.0,32,Grumpier Old Men (1995),1,3.5
3,4.0,47,Waiting to Exhale (1995),1,3.5
4,5.0,50,Father of the Bride Part II (1995),1,3.5


In [10]:
df_movies_ratings.keys()

Index(['movieId', 'movieId', 'title', 'userId', 'rating'], dtype='object')

In [11]:
df_movies_ratings.columns = ['movie', 'movieId', 'title','userId', 'rating']
df_movies_ratings.drop(['movie'], inplace=True, axis=1)

## Firefly Process

In this step, we transform the raw movie rating data into a Utility Matrix format, which is essential for feeding the data into recommendation algorithms. The utility matrix represents users (rows) and movies (columns), with ratings as the values. This format enables the recommender system to analyze user preferences and predict ratings for unrated movies.

In [13]:
# Utility Matrix
df = df_movies_ratings.head(1000000)
ratings_matrix_items = df.pivot_table(index=['userId'],
                       columns=['movieId'],values='rating').reset_index(drop=True)

ratings_matrix_items.fillna(0, inplace = True)

computes the pairwise similarity between movies based on user ratings using the Pearson Correlation method. The resulting movie similarity matrix can be used in content-based recommendation systems, where movies that are similar to those a user has already rated highly are suggested.

In [14]:
# Generate similarity pairwise matrix with pearson correlation

mov_sim = np.corrcoef(ratings_matrix_items)
matrix_item= pd.DataFrame(mov_sim)
movie_similarity = 1-matrix_item
matrix_item= pd.DataFrame(movie_similarity)
matrix_item.head()

,0,1,2,3,4,5,6,7,8,9,...,6733,6734,6735,6736,6737,6738,6739,6740,6741,6742
0,0.000000,0.903410,0.748229,0.975612,0.867153,0.974785,0.901583,0.946551,0.942228,0.867849,...,0.924012,0.993004,0.965932,0.799100,0.883604,0.818351,0.794642,0.902202,0.968087,0.826528
1,0.903410,0.000000,0.825633,0.938601,0.863663,0.909833,0.846962,0.900165,1.003017,0.901032,...,0.962302,0.914425,1.002947,0.776875,0.945511,0.894616,0.989281,0.979929,1.002395,0.866384
2,0.748229,0.825633,0.000000,0.947239,0.849530,0.940535,0.804709,0.919933,0.950993,0.729910,...,0.894890,0.923597,0.939282,0.728207,0.927431,0.689646,0.919671,0.953329,0.986959,0.738024
3,0.975612,0.938601,0.947239,0.000000,0.701151,0.961026,0.932020,0.694601,0.959021,0.976659,...,0.948713,1.002106,0.926705,0.853062,0.985716,0.951000,1.001747,0.974743,1.001638,0.828725
4,0.867153,0.863663,0.849530,0.701151,0.000000,0.767749,0.837575,0.699058,1.003167,0.864127,...,0.902637,0.816877,0.865106,0.709209,0.980502,0.876724,0.955968,0.945228,0.901913,0.666570


This code block implements a Firely Algorithm, a meta-heuristic optimization algorithm.

**Key Functions and Their Purposes:**

- **CalculateTime:** Calculates the time taken to travel a given distance at a specific speed.
- **GetDistance:** Retrieves the distance between two points from a given data matrix.
- **CalculateDistance:** Calculates the total distance of a given route.
- **PrintStatus:** Prints the current route and its corresponding time.
- **Init:** Initializes the data and the main route.
- **CalculateLightIntensity:** Calculates the light intensity of a firefly based on its route and speed.
- **GetLightIntensityFromAllFireflies:** Calculates the light intensity of all fireflies in a population.
- **InitialPopulation:** Generates an initial population of fireflies, each representing a potential solution.
- **OrderPopulation:** Sorts the population based on the light intensity of each firefly.
- **EvaluatePopulasi:** Evaluates the population and removes duplicate solutions.

In [15]:
def CalculateTime(space, speed):
    return 0 if speed == 0 else space/speed

def GetDistance(data, row, col):
    row -=1
    col -=1
#     print(row, col)
    return data.iloc[row][col]

def CalculateDistance(data, rute):
    totalDistance = 0
    for i in range(len(rute)-1):
        distance = GetDistance(data, rute[i], rute[i+1])
        if (distance == 0):
            return inf #No edge found
        totalDistance += distance
        if (rute[i+1] == a):
            return totalDistance #End Route
    return totalDistance

def PrintStatus(statusRoute, rute, time):
    print(statusRoute, rute, '\nTime: ', time, '\n\n')

def Init():
    #data = pd.read_csv("mov_rating.csv")
    data = matrix_item
    rute_utama =[]
    rute_utama.extend(range(0,a))
    space_rute_utama = CalculateDistance(data, rute_utama)
    speed_rute_utama = 1 # jeda antar userId
    #PrintStatus('Main Route: ', rute_utama, CalculateTime(space_rute_utama, speed_rute_utama))
    return data

def CalculateLightIntensity(data, speed, rute):
    space = CalculateDistance(data, rute)
    if (space == inf):
        return 0
    time = CalculateTime(space, speed)
    return 0 if time == 0 else 1/time

def GetLightIntensityFromAllFireflies(populasi, data, speed):
    lights = []
    for rute in populasi:
        lights.append(CalculateLightIntensity(data, speed, rute))
    return lights

def InitialPopulation(nPopulasi, nGen):
    populasi = []
    
    for i in range(nPopulasi):
        firefly=[]
        firefly.extend(np.random.choice(range(2,nGen+1), nGen-1, replace=False)) #Random permutasi, without replacement
        populasi.append(firefly)
    return populasi

def OrderPopulation(light, populasi):
    indeks = np.argsort(light, axis=0) #sorting ascending
    light_ = [light[idx] for idx in indeks]
    populasi_ = [populasi[idx] for idx in indeks]
    return light_, populasi_

def EvaluatePopulasi(populasi):
    return [k for k,v in groupby(sorted(populasi))]

**This code implements the main loop of the Firely Algorithm. Here's a breakdown:**

**1. Initialization:**

- Initializes the population of fireflies, data, and parameters like gamma, beta0, alpha, and theta.
- Calculates the initial light intensity of each firefly.
- Identifies the best firefly based on its light intensity.

**2. Main Loop:**

- Iterates until a solution is found or a maximum number of iterations is reached.
- Updates the randomness parameter alpha based on the iteration number.
- Combines the current population with temporary solutions.
- Evaluates the combined population to remove duplicates.
- Calculates the light intensity of each firefly in the updated population.
- Sorts the population based on light intensity.
- Checks if a better solution (higher light intensity) is found.
- Updates the best firefly if a better solution is found.

**For each firefly:**

- Compares it to brighter fireflies.
- Calculates the attractiveness based on the distance between the fireflies.
- Randomly selects an index to swap.
- Swaps elements in the firefly's route based on the attractiveness and randomness.
- Adds the modified firefly to the temporary population.

**3. Output:**

Prints the best firefly found, representing the optimal solution.

**In essence, the algorithm simulates the behavior of fireflies, where brighter fireflies attract dimmer ones. This movement and attraction lead to the optimization process, aiming to find the best solution to the given problem.**

In [16]:
### Main Program
a = 943
np.seterr(divide='ignore', invalid='ignore')
data = Init()
kecepatan_rute_alt = 1 # m/menit = 20km/jam
gamma = 0.01; # Absorption coefficient
beta0 = 1 # Attractiveness constant
alpha=1.0 # Randomness strength 0--1 (highly random)
theta=0.97 # Randomness reduction factor theta=10^(-5/tMax)
nPopulasi = 943
nGen = 943 # Dimension

lb=0
ub=943
scale = ub - lb

bestFirefly = []
solutionFound = False
populasi = InitialPopulation(nPopulasi, nGen)
tempPopulasi = []

light = GetLightIntensityFromAllFireflies(populasi, data, kecepatan_rute_alt)

light_, populasi_ = OrderPopulation(light, populasi)

# best firefly check
if (light_[len(light_)-1] > 0):
    solutionFound = True
    bestFirefly = populasi_[len(populasi_)-1]
    
while (solutionFound == False):
    alpha *= theta
    populasi.extend(tempPopulasi)
    populasi = EvaluatePopulasi(populasi) # Remove duplicate populasi
    light = GetLightIntensityFromAllFireflies(populasi, data, kecepatan_rute_alt)
    light_, populasi_ = OrderPopulation(light, populasi)
    
    # best firefly check
    if (light_[len(light_)-1] > 0):
        solutionFound = True
        bestFirefly = populasi_[len(populasi_)-1]
        break
    
    tempPopulasi = []
    
    # Movement populasi firefly
    for i in range(len(populasi_)):
        for j in range(i):
            if light_[j] >= light_[i]:
                selectedFirefly = populasi_[i]
                r=np.sqrt(np.sum([k**2 for k in ([a-b for a,b in zip(populasi_[i], populasi_[j])])])) #Euclidean distance
                beta = beta0*math.exp(-gamma*(r**2))
                indexForSwap = int(scale * np.random.uniform(0,1,1) + lb)
                if alpha*np.random.uniform(0,1,1) > 0.5:
                    index2ForSwap = indexForSwap + int(beta) + lb
                else:
                    index2ForSwap = indexForSwap - int(beta) - lb
                if (index2ForSwap > ub):
                    selectedFirefly[indexForSwap-1], selectedFirefly[ub-1] =  selectedFirefly[ub-1], selectedFirefly[indexForSwap-1]
                elif (index2ForSwap < lb):
                    selectedFirefly[indexForSwap-1], selectedFirefly[lb-1] =  selectedFirefly[lb-1], selectedFirefly[indexForSwap-1]
                else:
                    selectedFirefly[indexForSwap-1], selectedFirefly[index2ForSwap-1] =  selectedFirefly[index2ForSwap-1], selectedFirefly[indexForSwap-1]
                tempPopulasi.append(selectedFirefly)
                
print('Best Firefly: ', bestFirefly)

Best Firefly:  [608, 278, 312, 865, 230, 943, 723, 164, 824, 528, 788, 655, 596, 415, 701, 449, 3, 749, 574, 653, 107, 830, 729, 649, 868, 350, 832, 834, 584, 136, 38, 366, 704, 708, 877, 898, 915, 891, 243, 477, 692, 614, 64, 864, 922, 392, 486, 642, 526, 743, 109, 260, 265, 806, 636, 465, 87, 479, 803, 328, 47, 724, 390, 286, 744, 731, 414, 103, 68, 586, 32, 682, 210, 597, 659, 933, 748, 399, 348, 493, 562, 893, 445, 790, 705, 572, 149, 700, 710, 95, 923, 31, 90, 796, 782, 192, 816, 670, 209, 842, 646, 36, 454, 143, 25, 536, 48, 812, 717, 609, 502, 227, 397, 306, 792, 482, 581, 276, 473, 21, 66, 438, 384, 353, 7, 810, 866, 693, 679, 521, 739, 141, 859, 735, 490, 63, 70, 687, 5, 239, 148, 853, 492, 287, 44, 137, 529, 621, 410, 857, 211, 783, 299, 291, 152, 205, 98, 155, 285, 685, 658, 654, 826, 29, 852, 165, 245, 375, 35, 12, 333, 122, 379, 429, 828, 402, 305, 610, 114, 514, 418, 770, 168, 934, 34, 40, 338, 292, 848, 497, 747, 509, 320, 592, 413, 381, 151, 295, 62, 321, 406, 664, 712,

This code block creates a DataFrame to visualize the best firefly and its corresponding light intensity.

**Here's a breakdown:**

**1. Creating DataFrames for Best Firefly and Light Intensities:**

- `bf = pd.DataFrame(bestFirefly)`: Creates a DataFrame from the best firefly's route, labeling the column as 'user_id'.
- `lg = pd.DataFrame(sorted(light_, reverse=True))`: Creates a DataFrame from the sorted light intensities, labeling the column as 'weight'.

**2. Concatenating DataFrames:**

- `result = pd.concat([bf, lg], axis=1)`: Concatenates the two DataFrames horizontally, combining the 'user_id' and 'weight' columns into a single DataFrame.

**The resulting result DataFrame provides a clear view of the optimal sequence of user IDs (the best firefly's route) and their corresponding weights (light intensities). This information can be used to analyze the solution and gain insights into the optimization process.**

In [18]:
bf = pd.DataFrame(bestFirefly)
bf.rename(columns={0:'user_id'}, inplace=True)
lg = pd.DataFrame(sorted(light_, reverse=True))
lg.rename(columns={0:'weight'}, inplace=True)
result = pd.concat([bf, lg], axis =1)
result.head()

,user_id,weight
0,608.0,0.222100
1,278.0,0.211037
2,312.0,0.200313
3,865.0,0.180910
4,230.0,0.150132


In [19]:
result = result.dropna()
result

,user_id,weight
0,608.0,0.222100
1,278.0,0.211037
2,312.0,0.200313
3,865.0,0.180910
4,230.0,0.150132
...,...,...
937,458.0,0.001178
938,304.0,0.001177
939,846.0,0.001175
940,128.0,0.001174


In [20]:
result.describe()

,user_id,weight
count,942.000000,942.000000
mean,472.500000,0.006611
std,272.076276,0.017218
min,2.000000,0.001173
25%,237.250000,0.001593
50%,472.500000,0.002301
75%,707.750000,0.004682
max,943.000000,0.222100


## FA Weighting Result

Based on result FA produce 
1. The most weighted user_id : 943
2. Mean of Weight : 0.006611
3. Std of weight : 0.017218
4. Max of weight : 0.222100
5. Min of weight : 0.001173

### This merged DataFrame (`df_weight`) now contains information about movie ratings, user IDs, and the corresponding weights assigned by the Firely Algorithm. This enriched dataset can be further analyzed or used for recommendation purposes.

In [21]:
# Merge Rating and weight
df_weight = df.merge(result, left_on='userId', right_on='user_id')
df_weight.head()

,movieId,title,userId,rating,user_id,weight
0,3,Lord of Illusions (1995),2,4.0,2.0,0.002448
1,62,Love & Human Remains (1993),2,5.0,2.0,0.002448
2,70,Mad Love (1995),2,5.0,2.0,0.002448
3,110,Mallrats (1995),2,4.0,2.0,0.002448
4,242,Mighty Morphin Power Rangers: The Movie (1995),2,3.0,2.0,0.002448


This code defines a function experiment that evaluates a K-Nearest Neighbors (KNN) recommendation algorithm using the Firely Algorithm weights.

**Here's a breakdown:**

**1. Function Parameters:**

- k: The number of nearest neighbors to consider for recommendations.
- weight: Column name in df_weight containing the weights from the Firely Algorithm.
- test_size: Proportion of data to use for testing.

**2. Data Preparation:**

- Reader and Dataset: Creates a Surprise reader and a Dataset object from the df_weight DataFrame.
Selects columns relevant for recommendation: user IDs, movie ratings, and the weight column (str(weight)).
- Train-Test Split: Splits the data into training and testing sets using Surprise's train_test_split function.

**3. KNN Recommendation Algorithms:**

- Baseline: Creates a KNNBasic recommender with pearson_baseline similarity and no shrinkage.
- Weighted KNN: Creates a KNNBasic recommender with user-based similarity and the specified number of neighbors (k).

**4. Evaluation:**

- Testing Weighted KNN: Evaluates the weighted KNN model using the test function on the test set.
- Cross-Validation: Uses Surprise's cross_validate function to perform 5-fold cross-validation on the entire data with the weighted KNN model.
- Metrics: Evaluates the model's performance using two metrics: RMSE (Root Mean Squared Error) and MAE (Mean Absolute Error).
- Verbose Output: Sets verbose=True to display detailed information during cross-validation.

**Overall, this experiment aims to assess how the weights assigned by the Firely Algorithm influence the performance of a KNN-based recommendation system. It compares a baseline KNN and a weighted KNN to see if the weights improve the recommendation accuracy. You can run this experiment with different values of k and weight to find the optimal configuration for your recommendation system based on the Firely Algorithm's output.**

In [22]:
def experiment(k,weight,test_size):
    reader = Reader()
    data = Dataset.load_from_df(df_weight[[str(weight),'user_id','rating']], reader)
    trainset, testset = train_test_split(data, test_size=test_size, random_state=50)

    sim_options = {'name': 'pearson_baseline','shrinkage': 0}
    algo = KNNBasic(sim_options=sim_options)
    algo_knn = KNNBasic(k=k, sim_options=sim_options, user_based=True)
    prediction_knn = algo_knn.fit(trainset).test(testset)
    cv = cross_validate(algo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluate the performance of a KNN-based recommendation system

- k = 10
- Train = 80%
- User Based = True
- Sim = Pearson
- CV = 5
- Weight = YES

In [23]:
experiment(10,'weight',0.2)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0387  1.0343  1.0332  1.0400  1.0414  1.0375  0.0032  
MAE (testset)     0.7989  0.8029  0.8033  0.8073  0.8089  0.8043  0.0036  
Fit time      

In [24]:
k, weight, test_size = 10, 'weight', 0.2
reader = Reader()
data = Dataset.load_from_df(df_weight[[str(weight),'user_id','rating']], reader)
trainset, testset = train_test_split(data, test_size=test_size, random_state=50)

sim_options = {'name': 'pearson_baseline','shrinkage': 0}
algo = KNNBasic(sim_options=sim_options)
algo_knn = KNNBasic(k=k, sim_options=sim_options, user_based=True)
prediction_knn = algo_knn.fit(trainset).test(testset)
cv = cross_validate(algo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0377  1.0384  1.0447  1.0407  1.0351  1.0393  0.0032  
MAE (testset)     0.8097  0.8136  0.8184  0.8163  0.8124  0.8141  0.0030  
Fit time      

In [25]:
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


# Precision, Recall, and F1-Score at k

This function computes the Precision, Recall, F1-Score, and Accuracy for each user at the top-k recommendations. These metrics evaluate the recommendation system's ability to suggest relevant items to users.

- **Precision:** The proportion of recommended items in the top-k that are relevant.
- **Recall:** The proportion of relevant items that are successfully recommended in the top-k.
- **F1-Score:** The harmonic mean of Precision and Recall, balancing the two metrics.
- **Accuracy:** The proportion of correctly predicted items in relation to all items evaluated.

This function processes the predictions for all users, sorts them based on estimated values, and calculates the metrics for each user individually.

In [38]:
from collections import defaultdict

def precision_recall_f1_at_k(predictions, k=10, threshold=3.5):
    """Return precision, recall, and F1-score at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    f1_scores = dict()
    accuracy = {}
    
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items (true_r >= threshold)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k (est >= threshold)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        # F1 Score at K: Harmonic mean of precision and recall
        if precisions[uid] + recalls[uid] > 0:
            f1_scores[uid] = 2 * (precisions[uid] * recalls[uid]) / (precisions[uid] + recalls[uid])
        else:
            f1_scores[uid] = 0  # If both precision and recall are 0, F1 score is 0

        # Accuracy calculation: proportion of relevant and recommended items to all possible items
        accuracy[uid] = (n_rel + n_rec_k) / (n_rel_and_rec_k + n_rel + n_rec_k) if n_rel != 0 else 0

    return precisions, recalls, f1_scores, accuracy


In [37]:
# Example usage
precisions, recalls, f1_scores, acc = precision_recall_f1_at_k(predictions)

# Compute and print the average precision, recall, F1-score, and accuracy across all users
print(f"Average Precision: {sum(prec for prec in precisions.values()) / len(precisions)}")
print(f"Average Recall: {sum(rec for rec in recalls.values()) / len(recalls)}")
print(f"Average F1-Score: {sum(f1 for f1 in f1_scores.values()) / len(f1_scores)}")
print(f"Average Accuracy: {sum(ac for ac in acc.values()) / len(acc)}")

Average Precision: 0.4437195317375977
Average Recall: 0.41287915668627745
Average F1-Score: 0.3881704074083773
Average Accuracy: 0.8384726797691395
